In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [108 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,016 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [749 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,208 kB]
Get:12 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [40.9 kB]
Get:13 https://ppa.launchpadcon

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',')

# Show DataFrame
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Check column formats for the DataFrame
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [5]:
# 3. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [6]:
# 4. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query = """
SELECT
YEAR(date) as Year
, round(avg(Price),2) as AveragePrice
from home_sales
WHERE bedrooms = 4
GROUP BY Year
ORDER BY Year ASC
"""
spark.sql(query).show()


+----+------------+
|Year|AveragePrice|
+----+------------+
|2019|    300263.7|
|2020|   298353.78|
|2021|   301819.44|
|2022|   296363.88|
+----+------------+



In [7]:
# 5. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query1 = """
SELECT
date_built
, round(avg(Price),2) as AveragePrice
from home_sales
WHERE bedrooms = 3
and bathrooms = 3
GROUP BY date_built
ORDER BY date_built ASC
"""
spark.sql(query1).show()


+----------+------------+
|date_built|AveragePrice|
+----------+------------+
|      2010|   292859.62|
|      2011|   291117.47|
|      2012|   293683.19|
|      2013|   295962.27|
|      2014|   290852.27|
|      2015|    288770.3|
|      2016|   290555.07|
|      2017|   292676.79|
+----------+------------+



In [8]:
# 6. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query2 = """
SELECT
date_built
, round(avg(Price),2) as AveragePrice
from home_sales
WHERE bedrooms = 3
and bathrooms = 3
and floors = 2
and sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built ASC
"""
spark.sql(query2).show()


+----------+------------+
|date_built|AveragePrice|
+----------+------------+
|      2010|   285010.22|
|      2011|   276553.81|
|      2012|   307539.97|
|      2013|   303676.79|
|      2014|   298264.72|
|      2015|   297609.97|
|      2016|    293965.1|
|      2017|   280317.58|
+----------+------------+



In [9]:
# 7. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
query3 = """
SELECT
view
, round(avg(Price),2) as AveragePrice
from home_sales
WHERE price >= 350000
GROUP BY view
ORDER BY view ASC
"""
spark.sql(query3).show()
end_time = time.time()

print(f"---{end_time - start_time} seconds ---")

+----+------------+
|view|AveragePrice|
+----+------------+
|   0|   403848.51|
|   1|   401044.25|
|   2|   397389.25|
|   3|    398867.6|
|   4|   399631.89|
|   5|   401471.82|
|   6|   395655.38|
|   7|   403005.77|
|   8|   398592.71|
|   9|   401393.34|
|  10|   401868.43|
|  11|   399548.12|
|  12|   401501.32|
|  13|   398917.98|
|  14|   398570.03|
|  15|    404673.3|
|  16|   399586.53|
|  17|   398474.49|
|  18|   399332.91|
|  19|   398953.17|
+----+------------+
only showing top 20 rows

---0.9957189559936523 seconds ---


In [10]:
# 8. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [11]:
# 9. Check if the table is cached.
if spark.catalog.isCached("home_sales"):
  print("a table is cached")
else:
  print("a table is not cached")

a table is cached


In [12]:
# 10. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

cache_start_time = time.time()
query4 = """
SELECT
view
, round(avg(Price),2) as AveragePrice
from home_sales
WHERE price >= 350000
GROUP BY view
ORDER BY view ASC
"""
spark.sql(query4).show()

cache_end_time = time.time()

print(f"---{cache_end_time - cache_start_time} seconds ---")


+----+------------+
|view|AveragePrice|
+----+------------+
|   0|   403848.51|
|   1|   401044.25|
|   2|   397389.25|
|   3|    398867.6|
|   4|   399631.89|
|   5|   401471.82|
|   6|   395655.38|
|   7|   403005.77|
|   8|   398592.71|
|   9|   401393.34|
|  10|   401868.43|
|  11|   399548.12|
|  12|   401501.32|
|  13|   398917.98|
|  14|   398570.03|
|  15|    404673.3|
|  16|   399586.53|
|  17|   398474.49|
|  18|   399332.91|
|  19|   398953.17|
+----+------------+
only showing top 20 rows

---0.9722208976745605 seconds ---


In [13]:
# 11. Determine the runtime and compare it to uncached runtime.

print(f"The runtime for the uncached table was {end_time - start_time} seconds verse {cache_end_time - cache_start_time} seconds for the cached table")

The runtime for the uncached table was 0.9957189559936523 seconds verse 0.9722208976745605 seconds for the cached table


In [14]:
# 12. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy('date_built').mode('overwrite').parquet('parquet_home_sales')

In [15]:
# 13. Read the parquet formatted data.
pqdf = spark.read.parquet('parquet_home_sales')

In [16]:
# 14. Create a temporary table for the parquet data.
pqdf.createOrReplaceTempView("p_home_sales")

In [17]:
# 15. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

parquet_start_time = time.time()
query5 = """
SELECT
view
, round(avg(Price),2) as AveragePrice
from p_home_sales
WHERE price >= 350000
GROUP BY view
ORDER BY view ASC
"""
spark.sql(query5).show()

parquet_end_time = time.time()

print(f"---{parquet_end_time - parquet_start_time} seconds ---")

+----+------------+
|view|AveragePrice|
+----+------------+
|   0|   403848.51|
|   1|   401044.25|
|   2|   397389.25|
|   3|    398867.6|
|   4|   399631.89|
|   5|   401471.82|
|   6|   395655.38|
|   7|   403005.77|
|   8|   398592.71|
|   9|   401393.34|
|  10|   401868.43|
|  11|   399548.12|
|  12|   401501.32|
|  13|   398917.98|
|  14|   398570.03|
|  15|    404673.3|
|  16|   399586.53|
|  17|   398474.49|
|  18|   399332.91|
|  19|   398953.17|
+----+------------+
only showing top 20 rows

---1.0753638744354248 seconds ---


In [18]:
# 16. Determine the runtime and compare it to the cached version.
print(f"The runtime for the cached table was {cache_end_time - cache_start_time} seconds verse {parquet_end_time - parquet_start_time} seconds for the parquet table")

The runtime for the cached table was 0.9722208976745605 seconds verse 1.0753638744354248 seconds for the parquet table


In [19]:
# 17. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [20]:
# 18. Check if the home_sales is no longer cached
if spark.catalog.isCached("home_sales"):
  print("a table is till cached")
else:
  print("all clear")


all clear
